In [ ]:
from summer2 import CompartmentalModel
import matplotlib.pyplot as plt

In [ ]:
total_population = 1e5
latent_period = 8.
infectious_period = 7.
r0 = 13.
beta = r0 / infectious_period
infous_rate = 1. / latent_period
recovery_rate = 1. / infectious_period
infectious_seed = 1.

In [ ]:
def build_seir_model():
    compartments = (
        "Susceptible", 
        "Pre-infectious", 
        "Infectious", 
        "Immune"
    )
    seir_model = CompartmentalModel(
        times=(0., 120.),
        compartments=compartments,
        infectious_compartments=["Infectious"],
    )
    seir_model.set_initial_population(
        distribution={
            "Susceptible": total_population - infectious_seed, 
            "Infectious": infectious_seed
        }
    )
    seir_model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=beta,
        source="Susceptible",
        dest="Pre-infectious"
    )
    seir_model.add_transition_flow(
        name="progression", 
        fractional_rate=infous_rate,
        source="Pre-infectious", 
        dest="Infectious"
    )
    seir_model.add_transition_flow(
        name="recovery", 
        fractional_rate=recovery_rate, 
        source="Infectious", 
        dest="Immune"
    )
    seir_model.request_output_for_flow(
        name="incidence", 
        flow_name="progression"
    )
    return seir_model

In [ ]:
seir_model = build_seir_model()
seir_model.run()

## Fig. 4.3 (a)

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=120)

ax.plot(
    seir_model.times, 
    seir_model.get_outputs_df()["Susceptible"] / total_population, 
    color="0.5"
)
ax.set_ylim((0., 1.))
ax.set_ylabel("Proportion susceptible")
ax.set_xlim((0., 120.))

right_ax = ax.twinx()
right_ax.plot(
    seir_model.times, 
    seir_model.get_derived_outputs_df()["incidence"], 
    color="0", 
    linestyle="--",
)
right_ax.set_ylim((0., 8000.))

fig.suptitle("Measles");

## Fig. 4.3 (b)

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=120)

ax.plot(
    seir_model.times, 
    seir_model.get_outputs_df()["Susceptible"] / total_population, 
    color="0.5",
    label="Susceptible",
)
reproduction_number = r0 * seir_model.get_outputs_df()["Susceptible"] / total_population
ax.plot(
    seir_model.times,
    reproduction_number,
    color="0.",
    label="Rn",
)
ax.set_ylim((0., 13.))
ax.set_ylabel("Proportion susceptible")
ax.set_xlim((0., 120.))

right_ax = ax.twinx()
right_ax.plot(
    seir_model.times, 
    seir_model.get_derived_outputs_df()["incidence"], 
    color="0", 
    linestyle="--",
    label="Incidence",
)
right_ax.set_ylim((0., 8000.))

ax.legend()
right_ax.legend(loc=4)

fig.suptitle("Measles");